In [4]:
%matplotlib inline

from datetime import datetime

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from shapely.geometry import Polygon
from geopy.distance import vincenty
from sklearn import cross_validation
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

plt.style.use('ggplot')
pd.set_option('display.max_columns', 100)

In [5]:
import warnings
warnings.simplefilter(action = "ignore")

In [6]:
def feature_creation(df):
    
    df = df.copy()
    
    time_cols = ['departure', 'arrival']
    for col in time_cols:
        df[col] = pd.to_datetime(df[col])
        
    # departures: hour, day of week, weekend, peak time (6pm-2am), time (seconds)
    df['dep_hour'] = df.departure.apply(lambda x: x.hour)
    df['dep_dow'] = df.departure.apply(lambda x: x.weekday())
    df['dep_weekend'] = (df.dep_dow.isin([5, 6])) * 1
    df['dep_peak'] = (df.dep_hour.isin([18,19,20,21,22,23,0,1,2])) * 1
    start_time = np.array([pd.to_datetime('09/01/2012')] * len(df))
    df['dep_elapsed'] = ((df.departure - start_time) /\
                         np.timedelta64(1, 's')).astype(float)
    
    # length of trip, in seconds
    df['trip_time'] = ((df.arrival - df.departure) /\
                       np.timedelta64(1, 's')).astype(float)
    
    # dist ^2, ^3
    df['dist_sq'] = df.dist ** 2
    df['dist_cu'] = df.dist ** 3
    
    # dist x hour
    df['dist_hour'] = df.dist * df.dep_hour
    
    # distance traveled
    departures = zip(df.depy, df.depx)
    arrivals = zip(df.arry, df.arrx)
    coords = zip(departures, arrivals)
    distances = [vincenty(c[0], c[1]).miles for c in coords]
    df['dist_geopy'] = distances
    
    # sfo
    departures = zip(df.depx, df.depy)
    df['dep_sfo'] = np.array([Point(coords).within(sfo)
                              for coords in departures]) * 1
    arrivals = zip(df.arrx, df.arry)
    df['arr_sfo'] = np.array([Point(coords).within(sfo)
                              for coords in arrivals]) * 1
    df['sfo'] = df['dep_sfo'] + df['arr_sfo']
    
    return df

## Data

### SFO Polygon

In [7]:
sfo = Polygon((
    (-122.390356,37.623409),
    (-122.378511,37.618311),
    (-122.384777,37.608928),
    (-122.394991,37.613416),
    (-122.395163,37.615455),
    (-122.397566,37.618583),
    (-122.393703,37.620078),
    (-122.390442,37.623274)))

### Training Data

In [9]:
training = pd.read_csv('Taxi/Taxi_Train.csv')

In [11]:
training.head()

,0,9/1/12 0:11,9/1/12 0:20,13.2,1,-122.41354,37.802683,-122.421277,37.785395,38,30
0,1,9/1/12 0:23,9/1/12 0:31,10.65,1,-122.419700,37.786090,-122.435217,37.762177,30,94
1,2,9/1/12 0:45,9/1/12 0:49,9.00,1,-122.415120,37.774672,-122.407657,37.782615,10,11
2,3,9/1/12 0:41,9/1/12 0:54,13.95,2,-122.419392,37.806622,-122.415393,37.778115,40,10
3,4,9/1/12 1:09,9/1/12 1:13,7.35,1,-122.429722,37.797790,-122.418060,37.789032,45,32
4,5,9/1/12 1:40,9/1/12 1:52,11.75,1,-122.433323,37.784148,-122.411398,37.786965,77,7


In [10]:
training = feature_creation(training)

KeyError: 'departure'

### Split the Training Data

In [ ]:
train, dev = cross_validation.train_test_split(training, test_size=0.2)#, random_state=8675309)

## Training

### Predict Time

In [ ]:
gbr_time = GradientBoostingRegressor(learning_rate=0.01, max_depth=4, min_samples_leaf=5, n_estimators=1000)

In [ ]:
tf = ['num', 'dist', 'dep_hour', 'dist_hour', 'sfo', 'dep_elapsed', 'deptaz', 'arrtaz']

In [ ]:
time_train = train[tf].values

In [ ]:
time_label = train['trip_time'].values

In [ ]:
gbr_time.fit(time_train, time_label)

In [ ]:
time_dev = dev[tf].values

In [ ]:
times = gbr_time.predict(time_dev)

In [ ]:
plt.scatter(dev.trip_time, times, s=50, alpha=0.1, color='DimGray')
plt.plot(range(4000), range(4000), color='#348ABD', linewidth=2)
plt.xlim(0, 4000)
plt.ylim(0, 4000)

In [ ]:
print 'Best:', 241.25566775514835

In [ ]:
mean_squared_error(dev['trip_time'], times) ** 0.5

In [ ]:
dev['trip_time'] = times

### Predict Fare

In [ ]:
gbr = GradientBoostingRegressor(learning_rate=0.01, max_depth=4, min_samples_leaf=5, n_estimators=1000)

In [ ]:
features = ['dist', 'distcost', 'timecost', 'trip_time', 'dep_hour',
            'dist_hour', 'dist_sq', 'dist_cu', 'sfo', 'deptaz', 'arrtaz']

In [ ]:
features_train = train[features].values
label_train = train.fare.values

In [ ]:
gbr.fit(features_train, label_train)

In [ ]:
features_dev = dev[features].values

In [ ]:
dev['fare_predicted'] = gbr.predict(features_dev)

In [ ]:
plt.figure(figsize=(8,8))

plt.scatter(dev['fare'], dev['fare_predicted'], s=35, alpha=0.1, color='DimGray')
plt.plot(range(200), range(200), color='#348ABD', linewidth=2)

plt.xlim(-5, 200)
plt.ylim(-5, 200)

In [ ]:
print 'Best:', 4.3942468777917876

In [ ]:
mean_squared_error(dev['fare'], dev['fare_predicted']) ** 0.5

## Testing

In [ ]:
test = pd.read_csv('data/Taxi_Query_Routed.csv')

In [ ]:
test = feature_creation(test)

In [ ]:
time_test = test[tf].values

In [ ]:
test['trip_time'] = gbr_time.predict(time_test)

In [ ]:
features_test = test[features].values

In [ ]:
test['fare'] = gbr.predict(features_test)

In [ ]:
output = test[['id', 'fare']]
output.columns = ['ID', 'Fare']

Round Time?

In [ ]:
output.to_csv('data/results/_js00.csv', index=False)